In [7]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [8]:
import tensorflow as tf
from tensorflow.keras import layers, models

IMG_SIZE = 64 #tamanho imagem em px
CHANNELS = 1 #canais
BATCH_SIZE = 32 #tamanho do batch (lote) de treinamento

#instância de objeto que extrai o dataset de imagens a partir de sistema de pasta e subpastas (pasta principal = dataset, nome das subpastas = classe, imagens dentro das subpastas = amostras )
train_ds = tf.keras.utils.image_dataset_from_directory(
    r'C:\Users\paulo\OneDrive\Área de Trabalho\PDI_CNN\placas',
    validation_split=0.2, #20% pra validação - 80% pra treino
    subset="training",
    seed=123, #seed pra conectar a divisão de dataset de treino e teste
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    color_mode='grayscale'
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    r'C:\Users\paulo\OneDrive\Área de Trabalho\PDI_CNN\placas',
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    color_mode='grayscale'
)
print(f"Classes encontradas: {train_ds.class_names}")
print(f"Total de classes: {len(train_ds.class_names)}")

#normalização dos pixels das imagens
normalization_layer = layers.Rescaling(1./127.5, offset=-1) #objeto de normalização, pixels entre 1 e -1
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y)) #aplicação no dataset treino. map aplica a função lambda de normalização
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y)) #mesmo para teste/val


Found 36576 files belonging to 36 classes.
Using 29261 files for training.
Found 36576 files belonging to 36 classes.
Using 7315 files for validation.
Classes encontradas: ['Sample001', 'Sample002', 'Sample003', 'Sample004', 'Sample005', 'Sample006', 'Sample007', 'Sample008', 'Sample009', 'Sample010', 'Sample011', 'Sample012', 'Sample013', 'Sample014', 'Sample015', 'Sample016', 'Sample017', 'Sample018', 'Sample019', 'Sample020', 'Sample021', 'Sample022', 'Sample023', 'Sample024', 'Sample025', 'Sample026', 'Sample027', 'Sample028', 'Sample029', 'Sample030', 'Sample031', 'Sample032', 'Sample033', 'Sample034', 'Sample035', 'Sample036']
Total de classes: 36


In [9]:
import tf_keras as keras
import tensorflow_model_optimization as tfmot

#criação da cnn por camadas
def create_esp32_simple_cnn():
    inputs = keras.Input(shape=(64, 64, 1)) #camada de entrada com imagens 64x64x1
    
    x = keras.layers.Conv2D(16, (3, 3), strides=(2, 2), padding='same', activation='relu')(inputs) #camada convolucional (16 filtros de tamanho 3x3, com passos 2x2, padding de 0 para ir às bordas). 32x32x16 
    
    x = keras.layers.SeparableConv2D(32, (3, 3), strides=(2, 2), padding='same', activation='relu')(x) #convolução separável com 32 filtros (conceito de mobilenet), aqui as convoluções são feitas em plano e profundidade separadamente. 16x16x32
    
    x = keras.layers.SeparableConv2D(64, (3, 3), strides=(2, 2), padding='same', activation='relu')(x) #convolução separável com 64 filtros. 8x8x64
    
    x = keras.layers.GlobalAveragePooling2D()(x) #camada de pooling, aqui cada um dos 64 mapas de características tem sua média global retirada. gerando 1 número por mapa
    x = keras.layers.Dropout(0.2)(x) #desligamento de 20% dos neurônios aleatoriamente durante o treino para a rede não depender de neurônios específicos, evita overfiting
    
    outputs = keras.layers.Dense(36, activation='softmax')(x) #camada de saida com 36 neurônios (0-9, A-Z), ativação softmax retorna probabilidades
    
    return keras.Model(inputs=inputs, outputs=outputs) #retorno do modelo

base_model = create_esp32_simple_cnn() #criando modelo

q_aware_model = tfmot.quantization.keras.quantize_model(base_model) #faz o modelo reconhecer que será quantizado - simula 'erros' de arredondamento para que o backpropagation ajuste os pesos de modo a compensar a posterior quantização

q_aware_model.compile(
    optimizer='adam', #ajusta learning rate para cada neurônio individualmente baseado na necessidade
    loss='sparse_categorical_crossentropy', #definição da função de custo
    metrics=['accuracy'] #porcentagem de imagens que o modelo está acertando
)

early_stop = keras.callbacks.EarlyStopping( #parada antecipada
    monitor='val_loss', #valor monitorado 
    patience=5, # se por 5 épocas a perda não cair, ele para
    restore_best_weights=True # garante que o modelo final seja o melhor de todos
)

print("Iniciando Treinamento com QAT...")
q_aware_model.fit(train_ds, validation_data=val_ds, epochs=30, callbacks = [early_stop]) #treinamento do modelo com QAT, até 30 épocas.

Iniciando Treinamento com QAT...
Epoch 1/30
915/915 [==============================] - 50s 53ms/step - loss: 3.4401 - accuracy: 0.0600 - val_loss: 2.9261 - val_accuracy: 0.2211
Epoch 2/30
915/915 [==============================] - 11s 12ms/step - loss: 2.7577 - accuracy: 0.2231 - val_loss: 2.3604 - val_accuracy: 0.4230
Epoch 3/30
915/915 [==============================] - 13s 14ms/step - loss: 2.3464 - accuracy: 0.3559 - val_loss: 1.9723 - val_accuracy: 0.5340
Epoch 4/30
915/915 [==============================] - 12s 13ms/step - loss: 2.0377 - accuracy: 0.4429 - val_loss: 1.6694 - val_accuracy: 0.6144
Epoch 5/30
915/915 [==============================] - 12s 13ms/step - loss: 1.7788 - accuracy: 0.5205 - val_loss: 1.4176 - val_accuracy: 0.6749
Epoch 6/30
915/915 [==============================] - 11s 12ms/step - loss: 1.5572 - accuracy: 0.5815 - val_loss: 1.2274 - val_accuracy: 0.7154
Epoch 7/30
915/915 [==============================] - 13s 14ms/step - loss: 1.3794 - accuracy: 0.6283 -

In [10]:
import numpy as np

#um gerador de dados representativos, pega amostras para determinar os valores máximos e mínimos das ativações de cada camada, a fim de converter para 8 bits sem perder informação
def representative_data_gen():
    for input_value, _ in val_ds.take(100): #carrega 100 imagens reais do dataset, for input_value, _ (é uma tupla de iteração input_value = x, _ = y)
        yield [input_value] #retorno com yield é um iterador, itera sobre cada input_value/x/imagem

converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model) #TFLiteConverter tira dados que são necessários apenas para o treinamento e mantém apenas o necessário para inferência
converter.optimizations = [tf.lite.Optimize.DEFAULT] #Aqui é feito a quantização dos pesos, transforma kernels decimais em int8. converter.optimizations atributo lista
converter.representative_dataset = representative_data_gen #Quantização de valores de ativação. converter.representative_dataset atributo que espera função geradora/iteradora. aponta para a referência da função

converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8] #Força todas as operações para serem feitas em int8. converter.target_spec.supported_ops subobjeto que guarda regras de operações matemáticas           
converter.inference_input_type = tf.int8 #informa que a entrada será de pixels já convertidos para int8
converter.inference_output_type = tf.int8 #informa que a saída será de probabilidades já convertidas para int8

tflite_model_quant = converter.convert() #Finalizando conversão - String de bytes

with open("modelo_placa_int8.tflite", "wb") as f:
    f.write(tflite_model_quant) #escrita do arquivo .tflite no disco 

print("Modelo convertido com sucesso para modelo_placa_int8.tflite!")

INFO:tensorflow:Assets written to: C:\Users\paulo\AppData\Local\Temp\tmpzbm140qj\assets


INFO:tensorflow:Assets written to: C:\Users\paulo\AppData\Local\Temp\tmpzbm140qj\assets
C:\Users\paulo\AppData\Roaming\Python\Python312\site-packages\tensorflow\lite\python\convert.py:863: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


Modelo convertido com sucesso para modelo_placa_int8.tflite!


In [11]:
import os

def bin_to_header(filename, var_name='modelo_placa_int8'): 
    with open(filename, 'rb') as f: #abre arquivo no modo read bin
        data = f.read() #lê e guarda arquivo .tflite na var data
    
    with open(var_name + '.h', 'w') as f: #cria arquivo e adiciona extensão .h, modo write
        f.write(f'unsigned char {var_name}[] = {{\n  ') # Escreve a declaração do array em C++: tipo 'unsigned char' (1 byte) para garantir compatibilidade
        for i, byte in enumerate(data): #retorna iterador e byte
            f.write(f'0x{byte:02x}, ') #escreve o byte em hexadecimal seguido de vírgula, :02x força dois dígitos
            if (i + 1) % 12 == 0: #quebra linha a cada 12 bytes
                f.write('\n  ')
        f.write('\n};\n\n') #fecha array e quebra linha
        f.write(f'unsigned int {var_name}_len = {len(data)};\n') #guarda uma variável com tamanho em bytes do modelo
 
bin_to_header('modelo_placa_int8.tflite')
print("Arquivo modelo_placa_int8.h gerado com sucesso!")

Arquivo modelo_placa_int8.h gerado com sucesso!
